In [1]:
!pip install transformers
!pip install datasets
from google.colab import drive
# drive.mount('/content/drive')
# % cd /content/drive/MyDrive/TranslationArtifact_CodesAndGraphs

     |████████████████████████████████| 4.0 MB 5.0 MB/s 
     |████████████████████████████████| 895 kB 47.0 MB/s 
     |████████████████████████████████| 596 kB 51.4 MB/s 
     |████████████████████████████████| 6.6 MB 47.1 MB/s 
     |████████████████████████████████| 77 kB 8.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 325 kB 5.1 MB/s 
     |████████████████████████████████| 212 kB 69.8 MB/s 
     |████████████████████████████████| 136 kB 72.1 MB/s 
     |████████████████████████████████| 1.1 MB 62.2 MB/s 
     |████████████████████████████████| 127 kB 52.9 MB/s 
     |████████████████████████████████| 144 kB 73.0 MB/s 
     |████████████████████████████████| 94 kB 3.6 MB/s 
     |████████████████████████████████| 271 kB 71.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib

In [2]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AdamW
from transformers import get_linear_schedule_with_warmup, get_constant_schedule
import torch
import torch.nn as nn
import torch.nn.functional as F 
from datasets import load_dataset
import numpy as np
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support
import math
from tqdm import tqdm
import copy
import pickle
import os
import random

In [3]:
def find_answer(question_text, answer, ref_text, tokenizer) -> bool:
    model_input = tokenizer(question_text, ref_text, truncation=True, padding=True, return_token_type_ids=True, add_special_tokens=True)['input_ids'] #model input has both question text and reference text
    answer = tokenizer(answer, truncation=True, padding=True, return_token_type_ids=True, add_special_tokens=False)['input_ids']
    if len(model_input) > 512:
        return -1, -1
    for i in range(0, len(model_input) - len(answer) + 1):
        if answer == model_input[i: i + len(answer)]:
            return (i, i + len(answer))
    return -1, -1
# prepair input
def prepare_inputs(indexes, data, tokenizer):
    contexts = []
    questions = []
    answer_starts = []
    answer_ends = []
    for i in indexes:
        t = data[i]
        question = t['question']
        answer = t['answers']['text'][0]
        context = t['context']
        s, e = find_answer(question, answer, context, tokenizer)
        if s == -1:
            continue
        contexts.append(context)
        questions.append(question)
        answer_starts.append(s)
        answer_ends.append(e)
    return contexts, questions, answer_starts, answer_ends

In [4]:
def get_scheduler(optimizer, scheduler: str, warmup_steps: int, t_total: int):
    """
    Returns the correct learning rate scheduler
    """
    scheduler = scheduler.lower()
    if scheduler=='constantlr':
        return get_constant_schedule(optimizer)
    elif scheduler=='warmupconstant':
        return get_constant_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps)
    elif scheduler=='warmuplinear':
        return get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=t_total)
    elif scheduler=='warmupcosine':
        return get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=t_total)
    elif scheduler=='warmupcosinewithhardrestarts':
        return get_cosine_with_hard_restarts_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=t_total)
    else:
        raise ValueError("Unknown scheduler {}".format(scheduler))
        
def train(model, optimizer, scheduler, train_data, dev_data, batch_size, device, max_grad_norm, tokenizer, best_acc = -1):
    loss_fn = nn.CrossEntropyLoss()

    step_cnt = 0
    best_model_weights = None
    
    contexts, questions, answer_starts, answer_ends = train_data

    for pointer in tqdm(range(0, len(contexts), batch_size), desc='training',ascii = True,leave = True):
        model.train() # model was in eval mode in evaluate(); re-activate the train mode
        optimizer.zero_grad() # clear gradients first
        torch.cuda.empty_cache() # releases all unoccupied cached memory 
        # t = torch.cuda.get_device_properties(0).total_memory
        # r = torch.cuda.memory_reserved(0) 
        # a = torch.cuda.memory_allocated(0)
        # print(t,r,a)

        step_cnt += 1
        
        input = tokenizer(contexts[pointer:min(pointer + batch_size, len(contexts))], questions[pointer:min(pointer + batch_size, len(contexts))], return_tensors="pt",truncation=True, padding=True, return_token_type_ids=True, add_special_tokens=True)
        input.to(device)
        
        answer_start, answer_end = (answer_starts[pointer:min(pointer + batch_size, len(contexts))], answer_ends[pointer:min(pointer + batch_size, len(contexts))])
        true_labels1 = torch.LongTensor(np.array(answer_start)).to(device)
        true_labels2 = torch.LongTensor(np.array(answer_end)).to(device)
            
        output = model(**input)
        if output is None: continue
        pred_indicies1 = output['start_logits']
        pred_indicies2 = output['end_logits']
        # print(pred_indicies1.shape, true_labels1.shape)
        loss1 = loss_fn(pred_indicies1,true_labels1)
        loss2 = loss_fn(pred_indicies2,true_labels2)
        loss = loss1 + loss2

        # back propagate
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

        # update weights 
        optimizer.step()

        # update training rate
        scheduler.step()

        if step_cnt%int(len(contexts)/batch_size/10) == 0 or step_cnt == math.ceil(len(contexts)*1./batch_size):
            acc = evaluate(model,dev_data,device,tokenizer,mute=True)
            print('==> step {} dev acc: {}'.format(step_cnt,acc))
            if acc > best_acc:
                best_acc = acc
                best_model_weights = copy.deepcopy(model.cpu().state_dict())
                model.to(device)

    return best_model_weights
    
def evaluate(model, test_data, device, tokenizer, mute=False, batch_size=10):
    model.eval()
    contexts, questions, answer_starts, answer_ends = test_data
    all_labels = []
    all_predict = np.array([])
    with torch.no_grad():
        for pointer in range(0, len(contexts), batch_size):            
            input = tokenizer(contexts[pointer:min(pointer + batch_size, len(contexts))], questions[pointer:min(pointer + batch_size, len(contexts))], return_tensors="pt",truncation=True, padding=True, return_token_type_ids=True, add_special_tokens=True)
            input.to(device)

            answer_start, answer_end = (answer_starts[pointer:min(pointer + batch_size, len(contexts))], answer_ends[pointer:min(pointer + batch_size, len(contexts))])
            all_labels = all_labels + answer_start 
            all_labels = all_labels + answer_end
            
            outputs = model(**input)
            
            start_logits = outputs.start_logits
            end_logits = outputs.end_logits
            predict_start = [np.argmax(pp) for pp in start_logits.cpu()]
            predict_end = [np.argmax(pp) for pp in end_logits.cpu()]
            all_predict = np.concatenate((all_predict, predict_start), axis=None)
            all_predict = np.concatenate((all_predict, predict_end), axis=None)
    assert len(all_predict) == len(all_labels)


    acc = len([i for i in range(len(all_labels)) if all_predict[i]==all_labels[i]])*1./len(all_labels)

    if not mute:
        print('==>acc<==', acc)

    return acc

In [5]:
train_d = load_dataset('tydiqa', name = 'primary_task', split = 'train')
train_d[0]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/166916 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/18670 [00:00<?, ? examples/s]

Dataset tydiqa downloaded and prepared to /root/.cache/huggingface/datasets/tydiqa/primary_task/1.0.0/b8a6c4c0db10bf5703d7b36645e5dbae821b8c0e902dac9daeecd459a8337148. Subsequent calls will reuse this data.


{'annotations': {'minimal_answers_end_byte': [-1],
  'minimal_answers_start_byte': [-1],
  'passage_answer_candidate_index': [-1],
  'yes_no_answer': ['NONE']},
 'document_plaintext': '\ntransl.\n\nRas (dari bahasa Prancis race, yang sendirinya dari bahasa Latin radix, "akar") adalah suatu sistem klasifikasi yang digunakan untuk mengkategorikan manusia dalam populasi atau kelompok besar dan berbeda melalui ciri fenotipe, asal usul geografis, tampang jasmani dan kesukuan yang terwarisi. Di awal abad ke-20 istilah ini sering digunakan dalam arti biologis untuk menunjuk populasi manusia yang beraneka ragam dari segi genetik dengan anggota yang memiliki fenotipe (tampang luar) yang sama.[1] Arti "ras" ini masih digunakan dalam antropologi forensik (dalam menganalisa sisa tulang), penelitian biomedis dan kedokteran berdasarkan asal usul.[2]\nDi samping itu, di Amerika Serikat misalnya, penegak hukum menggunakan istilah "ras" dalam menentukan profil tersangka dan penggambaran kembali tampang

In [6]:
l = 'english'

train_d = load_dataset('tydiqa', name = 'secondary_task', split = 'train')
train_d = train_d.shuffle()
test_d = load_dataset('tydiqa', name = 'secondary_task', split = 'validation')

tokenizer=AutoTokenizer.from_pretrained('xlm-roberta-base')
languages = [(t['id'].split("-")[0]) for t in train_d]
# find target language indicies
indexes_train = [i for i,x in enumerate(languages) if x == l]
indexes_dev = indexes_train[-100:]
indexes_train = indexes_train[:-100]
languages = [(t['id'].split("-")[0]) for t in test_d]
indexes_test = [i for i,x in enumerate(languages) if x == l]

train_data = prepare_inputs(indexes_train, train_d, tokenizer)
dev_data = prepare_inputs(indexes_dev, train_d, tokenizer)
test_data = prepare_inputs(indexes_test, test_d, tokenizer)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/49881 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5077 [00:00<?, ? examples/s]

Dataset tydiqa downloaded and prepared to /root/.cache/huggingface/datasets/tydiqa/secondary_task/1.0.0/b8a6c4c0db10bf5703d7b36645e5dbae821b8c0e902dac9daeecd459a8337148. Subsequent calls will reuse this data.


Reusing dataset tydiqa (/root/.cache/huggingface/datasets/tydiqa/secondary_task/1.0.0/b8a6c4c0db10bf5703d7b36645e5dbae821b8c0e902dac9daeecd459a8337148)


Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

In [7]:
epoch_num = 100
batch_size = 5
warmup_percent = 0.2
max_grad_norm = 1
scheduler_setting = 'WarmupLinear'
device = 'cuda'
total_steps = math.ceil(epoch_num*len(train_data[0])*1./batch_size)
warmup_steps = int(total_steps*warmup_percent)

In [ ]:
model_path = 'ModelWeights'
iter = 0
# for i in range(0, epoch_num):
#     if os.path.exists('./model weights/' + model_path + str(i)):
#       iter = i
identifier = "xlm-roberta-base" if iter == 0 else './model weights/' + model_path + str(iter)
iter = iter + 1 if iter != 0 else iter

print(identifier)
model = AutoModelForQuestionAnswering.from_pretrained('xlm-roberta-base')
tokenizer=AutoTokenizer.from_pretrained('xlm-roberta-base')
model.to(device)
optimizer = AdamW(model.parameters(),lr=2e-5,eps=1e-6,correct_bias=False)
scheduler = get_scheduler(optimizer, scheduler_setting, warmup_steps=warmup_steps, t_total=total_steps) 
for i in range(0, epoch_num):
    best_weight = train(model, optimizer, scheduler, train_data, dev_data, batch_size, device, max_grad_norm, tokenizer, best_acc = -1)
    model.load_state_dict(best_weight)
    # model.save_pretrained(save_directory = './model weights/'+ model_path + str(i))

xlm-roberta-base


Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForQuestionAnswering: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForQuestionAnswering were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream tas

==> step 64 dev acc: 0.016483516483516484


training:  20%|#9        | 128/649 [00:54<08:40,  1.00it/s]

...

training: 100%|##########| 649/649 [05:01<00:00,  2.15it/s]

==> step 649 dev acc: 0.08241758241758242



training:  10%|9         | 63/649 [00:25<04:03,  2.41it/s]

==> step 64 dev acc: 0.0989010989010989


training:  16%|#6        | 105/649 [00:47<03:37,  2.50it/s]

In [ ]:
evaluate(model, test_data, device, tokenizer, mute=False, batch_size=10)